In [1]:
import sys
import os
import importlib
from collections import Counter
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# --- 1. Path Setup ---
# Absolute path to repo root (adjust if necessary)
repo_root = "/files/pixlball"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root) 

# --- 2. Project Module Imports ---
# Import all project modules using clean names
import src.config as config
import src.dataset as dataset
import src.train as train
import src.evaluate as evaluate
import src.data as data
import src.losses as losses
import src.model as model
import src.utils as utils

# --- 3. Module Reloading (CRITICAL for Notebook Development) ---
# Reload dependencies in order: Config/Utils -> Data/Losses/Model -> Train/Dataset/Evaluate
importlib.reload(config)
importlib.reload(utils)
importlib.reload(data)
importlib.reload(model)
importlib.reload(losses) 
importlib.reload(dataset)
importlib.reload(train)
importlib.reload(evaluate)

# --- 4. Direct Imports (For clean code in subsequent cells) ---
# Import essential classes and functions needed for the pipeline steps

# Configuration
from src.config import DEVICE 

# Data/Dataset Classes
from src.dataset import PitchDatasetMultiTask, TemporalPitchDataset, ContextPitchDatasetMultiTask, FusionPitchDataset

# Training Functions
from src.train import train_model_base, train_model_lstm, train_model_context, train_model_lstm_fused

# Evaluation/Helpers
from src.evaluate import evaluate_model_base, evaluate_model_lstm, evaluate_model_context, evaluate_model_lstm_fused
from src.losses import get_model_criteria
from src.model import TinyCNN_MultiTask, HybridCNN_LSTM, TinyCNN_LSTM_Fused
from src.utils import get_sequence_lengths

# --- Final Check ---
print(f"Using device: {DEVICE}")

Using device: cpu


In [2]:
data_events = pd.read_parquet(os.path.join(repo_root, "data", "events_data.parquet"), engine="fastparquet")
data_360 = pd.read_parquet(os.path.join(repo_root, "data", "sb360_data.parquet"), engine="fastparquet")

In [3]:
admin_events = [
        'Starting XI', 'Half Start', 'Half End', 'Player On', 'Player Off',
        'Substitution', 'Tactical Shift', 'Referee Ball-Drop', 'Injury Stoppage',
        'Bad Behaviour', 'Shield'
    ]

cleaned_df = data.drop_events(data_events, rows_to_drop=admin_events)

1278 events.


In [4]:
# -----------------------------
# Example usage
# -----------------------------
columns_to_drop = ['clearance_body_part',
                   'clearance_head',
                   'clearance_left_foot',
                   'clearance_other',
                   'clearance_right_foot',
                   'shot_technique',
                   'substitution_replacement_id',
                   'substitution_replacement',
                   'substitution_outcome',
                   'shot_saved_off_target',
                   'pass_miscommunication',
                   'goalkeeper_shot_saved_off_target',
                   'goalkeeper_punched_out',
                   'shot_first_time',
                   'shot_first_time',
                   'shot_body_part',
                   'related_events',
                   'pass_shot_assist', 
                   'pass_straight', 
                   'pass_switch', 
                   'pass_technique', 
                   'pass_through_ball',
                   'goalkeeper_body_part',
                   'goalkeeper_end_location', 
                   'goalkeeper_outcome', 
                   'goalkeeper_position', 
                   'goalkeeper_technique', 
                   'goalkeeper_type', 
                   'goalkeeper_penalty_saved_to_post', 
                   'goalkeeper_shot_saved_to_post', 
                   'goalkeeper_lost_out', 
                   'goalkeeper_Clear', 
                   'goalkeeper_In Play Safe',
                   'shot_key_pass_id',
                   'shot_one_on_one',
                   'shot_end_location',
                   'shot_type',
                   'pass_angle',
                   'pass_body_part',
                   'pass_type',
                   'pass_length',
                   'pass_outswinging',
                   'pass_inswinging',
                   'pass_cross', 
                   'pass_cut_back', 
                   'pass_deflected', 
                   'pass_goal_assist', 
                   'pass_recipient', 
                   'pass_recipient_id', 
                   'pass_assisted_shot_id', 
                   'pass_no_touch', 
                   'pass_end_location', 
                   'pass_aerial_won',
                   'pass_height',
                   'substitution_outcome_id',
                   'tactics',
                   'block_deflection',
                   'dribble_no_touch',
                   'shot_open_goal', 
                   'shot_saved_to_post',
                   'shot_redirect', 
                   'shot_follows_dribble',
                   'period',
                   'injury_stoppage_in_chanin',
                   'block_save_block',
                   'ball recovery_offensive',


                   ]
cleaned_df = data.drop_columns(cleaned_df, columns_to_drop)

# add lookahead outcome
df_with_targets = data.assign_lookahead_outcomes(cleaned_df, lookahead=6)


counts of each outcome nn_target
Keep Possession    71251
Lose Possession    28252
Shot                4830
Name: count, dtype: int64


# Prepare 360 Data

In [5]:
df_360 = data.assign_grid_cells(data_360)
nn_final = data.aggregate_nn_layers_vectorized(df_360)

# Finalize NN Df

In [6]:
nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True ) # adjust cols depending on model

# Neural Network final Data Prep

In [7]:
context_cols = [
    'under_pressure', 
    'counterpress', 
    'dribble_nutmeg'
]

# Impute NaN values with 0.0 (float)
# This assumes NaN means the event was NOT under pressure, NOT a counterpress, etc.
nn_dataset[context_cols] = nn_dataset[context_cols].fillna(0.0)


target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Check
print(nn_dataset[['nn_target', 'nn_target_int']].head())

         nn_target  nn_target_int
0  Keep Possession              0
1  Keep Possession              0
2  Keep Possession              0
3  Keep Possession              0
4  Keep Possession              0


# The LSTM

In [9]:
print(f"Using device: {DEVICE}")

layer_columns = ["ball_layer", "teammates_layer", "opponents_layer"]

# Get your existing targets
event_targets = nn_dataset['nn_target_int'].values
goal_flags = nn_dataset['goal_flag'].values.astype(np.float32) # Ensure targets are float

event_counts = Counter(event_targets)
total_events = len(event_targets)

# Using inverse frequency: total / count
class_weights_event = torch.tensor(
    [total_events / event_counts.get(c, 1) for c in range(len(event_counts))],
    dtype=torch.float32
).to(DEVICE)

# B. Goal Positive Weight (Binary) - For BCEWithLogitsLoss
goal_counts = Counter(goal_flags)
# CRITICAL: pos_weight = (Number of Negative Samples) / (Number of Positive Samples)
# Here: pos_weight = (Number of No Goals) / (Number of Goals)
goal_pos_weight = torch.tensor(
    goal_counts.get(0.0, 1) / goal_counts.get(1.0, 1),
    dtype=torch.float32
).to(DEVICE)

print(f"Goal Positive Weight (0/1 ratio): {goal_pos_weight.item():.2f}")



Using device: cpu
Goal Positive Weight (0/1 ratio): 168.51


In [10]:
# -----------------------------
# Prepare Sequential Dataset
# -----------------------------
print("Preparing 5D Sequential Dataset for LSTM...")

nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag', 'possession']) # adjust cols depending on model

target_map = {"Keep Possession": 0, "Lose Possession": 1, "Shot": 2}

# Apply mapping
nn_dataset['nn_target_int'] = nn_dataset['nn_target'].map(target_map)

# Check
print(nn_dataset[['nn_target', 'nn_target_int']].head())

windows = data.build_temporal_windows_with_mask(nn_dataset)

# Assuming 'windows' variable holds the output of data.build_temporal_windows_with_mask()
# Shape: (Num_Events, T, 4, H, W)
temporal_dataset = TemporalPitchDataset(
    windows=windows, 
    event_labels=event_targets, 
    goal_flags=goal_flags
)



Preparing 5D Sequential Dataset for LSTM...
         nn_target  nn_target_int
0  Keep Possession              0
1  Keep Possession              0
2  Keep Possession              0
3  Keep Possession              0
4  Keep Possession              0


# LSTM CNN with Context

In [12]:
# Assuming you previously defined these:
NUM_CONTEXT_FEATURES = 3 
FINAL_CONTEXTUAL_FEATURES = ['under_pressure', 'counterpress', 'dribble_nutmeg']
nn_dataset = data.prepare_nn_dataset(df_with_targets, nn_final, target_cols=['nn_target', 'goal_flag'], context_cols = True, keep_context_ids = True ) # adjust cols depending on model
# 1. Prepare 1D Context Features (from the static context_df)
# Assuming 'context_df' is defined and NaN-imputed in a previous cell
context_df = nn_dataset[FINAL_CONTEXTUAL_FEATURES].copy().fillna(0.0)
context_features_array = context_df.values # Shape: (N_events, 3)

# 2. Determine the Sequence Length (T)
# CRITICAL FIX: The list 'windows' contains 'N' items. The length of the first item is T.
if isinstance(windows, list) and len(windows) > 0:
    # Use array conversion to safely get the shape of the first sequence item
    T = np.array(windows[0]).shape[0] 
else:
    # If windows is already a NumPy array, use its shape
    T = windows.shape[1] 

print(f"Sequence Length (T): {T}")

# 3. Create the Context Sequence (T-frames for each event)
# Repeat the static context features (N, 3) across the T dimension: (N, T, 3)
# context_features_array is (N, 3). We insert a T-dimension and repeat the feature vector T times.
context_sequence = np.repeat(context_features_array[:, np.newaxis, :], T, axis=1)

print(f"Context Sequence Shape: {context_sequence.shape} (N, T, F)")

# --- 4. Create Fused Dataset Instance (CRITICAL: Reload and Import) ---
# ... rest of your import and dataset creation code ...

fused_dataset = FusionPitchDataset(
    windows=windows, # The FusionPitchDataset class will handle the list to array conversion internally
    contextual_features=context_sequence, 
    event_labels=event_targets, 
    goal_flags=goal_flags
)

print(f"Fused Dataset Size: {len(fused_dataset)}")

Sequence Length (T): 4
Context Sequence Shape: (90690, 4, 3) (N, T, F)
Fused Dataset Size: 90690


In [13]:
# Assuming class_weights_event and goal_pos_weight are defined

print("Starting training for TinyCNN_LSTM_Fused Model...")

fused_lstm_model = train_model_lstm_fused(
    dataset=fused_dataset, 
    event_class_weights=class_weights_event,
    goal_pos_weight=goal_pos_weight,
    num_context_features=NUM_CONTEXT_FEATURES # Pass the number of 1D features
)

print("\nTinyCNN_LSTM_Fused Training complete.")

Fused LSTM Epoch 1:   0%|          | 13/9069 [00:00<01:14, 122.04it/s, event_loss=1.08, loss=4.21, shot_loss=0.626]

Starting training for TinyCNN_LSTM_Fused Model...


Fused LSTM Epoch 2: 100%|██████████| 9069/9069 [01:13<00:00, 123.58it/s, event_loss=0.907, loss=0.907, shot_loss=0]  


TinyCNN_LSTM_Fused Training complete.


In [14]:
print("\nEvaluating Fused LSTM Model on training data...")

fused_metrics = evaluate_model_lstm_fused(
    model=fused_lstm_model, 
    dataset=fused_dataset
)


Evaluating Fused LSTM Model on training data...
Event Accuracy: 0.6178409968022935
Event Balanced Accuracy: 0.3699842548543841
Event Confusion Matrix:
 [[47874 14785     0]
 [15426  8158     0]
 [ 3170  1277     0]]
              precision    recall  f1-score   support

           0       0.72      0.76      0.74     62659
           1       0.34      0.35      0.34     23584
           2       0.00      0.00      0.00      4447

    accuracy                           0.62     90690
   macro avg       0.35      0.37      0.36     90690
weighted avg       0.59      0.62      0.60     90690

Goal Accuracy: 0.12030582415111311
Goal Balanced Accuracy: 0.5
Goal Confusion Matrix:
 [[   0 3912]
 [   0  535]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3912
         1.0       0.12      1.00      0.21       535

    accuracy                           0.12      4447
   macro avg       0.06      0.50      0.11      4447
weighted avg    

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
